In [30]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
import time
import warnings

from tqdm import tqdm

# Suppress unnecessary FutureWarning
warnings.filterwarnings('ignore', category=FutureWarning)

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
import tqdm

In [5]:
topics = pd.read_csv('/kaggle/input/sampled_train_topics.csv')
qrels = pd.read_csv('/kaggle/input/sampled_train_qrels.csv')

In [6]:
!curl -s "https://get.sdkman.io" | bash && source "$HOME/.sdkman/bin/sdkman-init.sh" && sdk install java 11.0.22-amzn < /dev/null


                                -+syyyyyyys:
                            `/yho:`       -yd.
                         `/yh/`             +m.
                       .oho.                 hy                          .`
                     .sh/`                   :N`                `-/o`  `+dyyo:.
                   .yh:`                     `M-          `-/osysoym  :hs` `-+sys:      hhyssssssssy+
                 .sh:`                       `N:          ms/-``  yy.yh-      -hy.    `.N-````````+N.
               `od/`                         `N-       -/oM-      ddd+`     `sd:     hNNm        -N:
              :do`                           .M.       dMMM-     `ms.      /d+`     `NMMs       `do
            .yy-                             :N`    ```mMMM.      -      -hy.       /MMM:       yh
          `+d+`           `:/oo/`       `-/osyh/ossssssdNMM`           .sh:         yMMN`      /m.
         -dh-           :ymNMMMMy  `-/shmNm-`:N/-.``   `.sN            /N-         `NMMy      .m/
  

In [7]:
pip install python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 9.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
%env JAVA_HOME=/root/.sdkman/candidates/java/current
import pyterrier as pt
if not pt.started():
    pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

env: JAVA_HOME=/root/.sdkman/candidates/java/current
terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [9]:
!wget https://trec.nist.gov/trec_eval/trec_eval_latest.tar.gz
!tar -xvzf trec_eval_latest.tar.gz

--2024-05-29 17:06:04--  https://trec.nist.gov/trec_eval/trec_eval_latest.tar.gz
Resolving trec.nist.gov (trec.nist.gov)... 132.163.4.175, 2610:20:6b01:4::175
Connecting to trec.nist.gov (trec.nist.gov)|132.163.4.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 181743 (177K) [application/x-gzip]
Saving to: 'trec_eval_latest.tar.gz.2'

trec_eval_latest.ta 100%[===================>] 177.48K   309KB/s    in 0.6s    

2024-05-29 17:06:06 (309 KB/s) - 'trec_eval_latest.tar.gz.2' saved [181743/181743]

trec_eval-9.0.7/
trec_eval-9.0.7/CHANGELOG
trec_eval-9.0.7/Makefile
trec_eval-9.0.7/README
trec_eval-9.0.7/README.windows.md
trec_eval-9.0.7/bpref_bug
trec_eval-9.0.7/common.h
trec_eval-9.0.7/convert_zscores.c
trec_eval-9.0.7/form_prefs_counts.c
trec_eval-9.0.7/form_res_rels.c
trec_eval-9.0.7/form_res_rels_jg.c
trec_eval-9.0.7/formats.c
trec_eval-9.0.7/functions.h
trec_eval-9.0.7/get_prefs.c
trec_eval-9.0.7/get_qrels.c
trec_eval-9.0.7/get_qrels_jg.c
trec_eval-9.0.7

In [10]:
!cd trec_eval-9.0.7 && make

make: 'trec_eval' is up to date.


In [11]:
dataset = pt.get_dataset('irds:dpr-w100')

In [ ]:
# Remove stopwords
start_time = time.time()

indexer =  pt.IterDictIndexer("./indexes/stopwords")
indexer.setProperty("termpipelines", "Stopwords")
index_ref = indexer.index(dataset.get_corpus_iter(), fields=['text', 'title'])
index_ref = pt.IndexRef.of("./indexes/stopwords/data.properties")
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

end_time = time.time()
print(end_time - start_time)
len((index.getLexicon()))

In [12]:
index_ref = pt.IndexRef.of("./indexes/stopwords/data.properties")
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

17:06:08.627 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
17:06:13.235 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup file directly from disk (SLOW) - try index.meta.index-source=fileinmem in the index properties file. 160.3 MiB of memory would be required.
17:06:13.271 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 470.4 MiB of memory would be required.
Number of documents: 21015324
Number of terms: 4630820
Number of postings: 1030301407
Number of fields: 

In [ ]:
search_times = []

# randomly selected 100 items
ix_range = random.sample(range(len(index.getLexicon())), 100)

# timimg
for ix, kv in enumerate(index.getLexicon()):
    start_time = time.time()
    if ix in ix_range:
        pointer = index.getLexicon()[kv.getKey()]
    end_time = time.time()
    search_time = end_time - start_time
    search_times.append(search_time)
    
print(sum(search_times)/100)

In [13]:
topics["qid"] = topics["qid"].astype(str)
qrels["qid"] = qrels["qid"].astype(str)
qrels["docno"] = qrels["docno"].astype(str)

In [ ]:
qrels.to_csv('qrels.txt', sep=' ', index=False, header=False)

In [52]:
topics_sample = topics.sample(2000, random_state=34)

In [37]:
BM25 = pt.BatchRetrieve(index, wmodel="BM25",
                             controls={"bm25.b" : 0.05, "bm25.k_1": 1}
                       ).parallel(4)

In [ ]:
results = rm3_pipe.transform(topics)

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


09:18:37.899 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
09:18:37.909 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
09:18:38.158 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based mod

In [31]:
bm25_results = pd.read_csv('./results02.txt', sep=' ', header=None,
                           names=['qid','docid','docno','rank','score','query'])

In [21]:
bm25_results = bm25_results.drop(columns=['docid'])
bm25_results['tag']='arc616GZX124'

In [ ]:
bm25_results_0 = bm25_results.drop(columns=['docid','query'])
bm25_results_0['q0']='Q0'
cols = bm25_results_0.columns.tolist()
cols.insert(1, cols.pop(cols.index('q0')))
bm25_results_0 = bm25_results_0[cols]
bm25_results_0['tag']='arc616GZX124'

In [48]:
print(bm25_results.head())

     qid    docid    docno  rank      score  \
0  56250  5221006  5221007     0  73.611173   
1  56250   531917   531918     1  73.320892   
2  56250  5220980  5220981     2  71.971148   
3  56250  5221008  5221009     3  70.329160   
4  56250  3462200  3462201     4  67.949786   

                                          query  
0  who played boss hogg in the dukes of hazzard  
1  who played boss hogg in the dukes of hazzard  
2  who played boss hogg in the dukes of hazzard  
3  who played boss hogg in the dukes of hazzard  
4  who played boss hogg in the dukes of hazzard  


In [33]:
bm25_top100 = bm25_results[bm25_results['rank'] <= 99]

In [47]:
print(bm25_top100.head())

     qid    docid    docno  rank      score  \
0  56250  5221006  5221007     0  73.611173   
1  56250   531917   531918     1  73.320892   
2  56250  5220980  5220981     2  71.971148   
3  56250  5221008  5221009     3  70.329160   
4  56250  3462200  3462201     4  67.949786   

                                          query  
0  who played boss hogg in the dukes of hazzard  
1  who played boss hogg in the dukes of hazzard  
2  who played boss hogg in the dukes of hazzard  
3  who played boss hogg in the dukes of hazzard  
4  who played boss hogg in the dukes of hazzard  


In [58]:
bm25_top100.to_csv('./bm25_top100.txt', sep=' ', index=False, header=False)

OSError: Cannot save file into a non-existent directory: 'D:/info_retrieval/project'

In [63]:
import os
os.chdir('/kaggle/working')
from IPython.display import FileLink
FileLink('bm25_top100.txt')

/kaggle/working/bm25_top100.txt

In [ ]:
results.to_csv('results02.txt', sep=' ', index=False, header=False)

In [ ]:
%%time

!./trec_eval-9.0.7/trec_eval -q -m all_trec qrels.txt results02.txt

In [ ]:
print(qrels.head())

In [ ]:
rm3_pipes = []
lambda_values = [0.4, 0.5, 0.6, 0.7, 0.8]
term_values = [20]
doc_values = [1, 3, 5]

for term in term_values:
    for doc in doc_values:
        rm3_pipe = BM25 >> pt.rewrite.RM3(index,
                                          fb_terms = term,
                                          fb_docs = doc,
                                          fb_lambda = 0.8) >> BM25
        rm3_pipes.append(rm3_pipe)

In [39]:
rm3 = pt.rewrite.RM3(index, fb_terms = 15, fb_docs = 3, fb_lambda = 0.8)

In [40]:
rm3_bm25 = rm3 >> BM25

In [44]:
rm3_pipeline = BM25 >> rm3

In [49]:
results = rm3_pipeline.transform(bm25_results)

18:06:04.405 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
18:06:05.539 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
18:06:07.108 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based mod

In [50]:
print(results)

Empty DataFrame
Columns: [qid, query_0, query]
Index: []


In [ ]:
from pyterrier.measures import *

pt.Experiment(
    retr_systems = [rm3_pipe],
    topics = topics,
    qrels = qrels,
    eval_metrics = [NDCG@5, NDCG@10, NDCG@20,
                   RR@5, RR@10, RR@20,
                   P@5, P@10, P@20,
                   R@5, R@10, R@20]
)

In [ ]:
from pyterrier.measures import *

pt.Experiment(
    retr_systems = rm3_pipes,
    topics = topics_sample,
    qrels = qrels,
    eval_metrics = [NDCG@5, NDCG@10, NDCG@20,
                   RR@5, RR@10, RR@20,
                   P@5, P@10, P@20,
                   R@5, R@10, R@20]
)

In [ ]:
BM25_models = []
b_values = [0]
k1_values = [0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8]

for b in b_values:
    for k1 in k1_values:
        BM25 = pt.BatchRetrieve(index, wmodel="BM25",
                             controls={"bm25.b" : b, "bm25.k_1": k1}
                       ).parallel(4)
        BM25_models.append(BM25)

In [ ]:
from pyterrier.measures import *

pt.Experiment(
    retr_systems = [BM25],
    topics = topics,
    qrels = qrels,
    eval_metrics = [NDCG@5, NDCG@10, NDCG@20,
                   RR@5, RR@10, RR@20,
                   P@5, P@10, P@20,
                   R@5, R@10, R@20]
)

In [ ]:
from pyterrier.measures import *

pt.GridSearch(
    BM25,
    params = {BM25: {"bm25.b": [0.45, 0.55, 0.65, 0.85, 0.95, 1.05],
            "bm25.k_1": [0.6, 0.8, 1.0, 1.4, 1.6, 1.8],
    }},
    batch_size = 50,
    topics = topics,
    qrels = qrels,
    metric = NDCG@10
)

In [ ]:
from pyterrier.measures import *

tuned_BM25, _ = pt.KFoldGridSearch(
    BM25,
    params = {BM25: {"bm25.b"  : [0.45, 0.55, 0.65, 0.85, 0.95, 1.05],
            "bm25.k_1": [0.6, 0.8, 1.0, 1.4, 1.6, 1.8]
    }},
    batch_size = 50,
    topics_list = topics_list,
    qrels = qrels,
    metric = NDCG@10
)
#pt.Experiment([pl2, tuned_pl2], all_topics, qrels, ["map"])

In [64]:
DLM = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"dirichletlm.mu": 400}).parallel(4)

In [65]:
results_dlm = DLM.transform(topics)

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
/opt/conda/lib/python3.10/site-packages/pyterrier/parallel.py:32: UserWarning: Avoiding reinit of PyTerrier
  warn("Avoiding reinit of PyTerrier")
/opt/conda/lib/python3.10/site-packages/pyterrier/parallel.py:32: UserWarning: Av

19:43:58.381 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
19:43:58.641 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
19:43:58.700 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based mod

/opt/conda/lib/python3.10/site-packages/pyterrier/parallel.py:32: UserWarning: Avoiding reinit of PyTerrier
  warn("Avoiding reinit of PyTerrier")


19:43:59.168 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
19:44:17.405 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup file directly from disk (SLOW) - try index.meta.index-source=fileinmem in the index properties file. 160.3 MiB of memory would be required.
19:44:17.477 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 470.4 MiB of memory would be required.
19:44:17.718 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup

In [66]:
print(results_dlm.head())

     qid    docid    docno  rank      score  \
0  56250  5221006  5221007     0  42.116501   
1  56250  5220980  5220981     1  41.134714   
2  56250   531917   531918     2  40.798912   
3  56250  5221008  5221009     3  40.070150   
4  56250  3462200  3462201     4  39.355761   

                                          query  
0  who played boss hogg in the dukes of hazzard  
1  who played boss hogg in the dukes of hazzard  
2  who played boss hogg in the dukes of hazzard  
3  who played boss hogg in the dukes of hazzard  
4  who played boss hogg in the dukes of hazzard  


In [67]:
dlm_top100 = results_dlm[results_dlm['rank'] <= 99]

In [68]:
dlm_top100 = dlm_top100.sort_values(by=['qid', 'rank'], ascending=[True, True])

In [70]:
dlm_top100 = dlm_top100.drop(columns=['docid','query'])
dlm_top100['q0']='Q0'
cols = dlm_top100.columns.tolist()
cols.insert(1, cols.pop(cols.index('q0')))
dlm_top100 = dlm_top100[cols]
dlm_top100['tag']='arc616GZX125'

In [71]:
print(dlm_top100.head())

        qid  q0     docno  rank      score           tag
5526000   1  Q0   8685225     0  16.043088  arc616GZX125
5526001   1  Q0   6201542     1  15.889710  arc616GZX125
5526002   1  Q0  10017362     2  15.839638  arc616GZX125
5526003   1  Q0  19416619     3  15.624318  arc616GZX125
5526004   1  Q0   8685227     4  15.512973  arc616GZX125


In [72]:
dlm_top100.to_csv('./dlm_top100.txt', sep=' ', index=False, header=False)

In [73]:
import os
os.chdir('/kaggle/working')
from IPython.display import FileLink
FileLink('dlm_top100.txt')

/kaggle/working/dlm_top100.txt

In [55]:
DLM_models = []
mu_values = [100, 200, 300, 400]

for mu in mu_values:
    DLM = pt.BatchRetrieve(index, wmodel="DirichletLM",
                            controls={"dirichletlm.mu": mu}
                    ).parallel(4)
    DLM_models.append(DLM)

In [56]:
from pyterrier.measures import *
pt.Experiment(
    retr_systems=DLM_models,
    topics=topics_sample,
    qrels=qrels,
    eval_metrics=[NDCG@5, NDCG@10, NDCG@20,
                   RR@5, RR@10, RR@20,
                   P@5, P@10, P@20,
                   R@5, R@10, R@20]
)

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.
/opt/conda/lib/python3.10/site-packages/pyterrier/parallel.py:32: UserWarning: Avoiding reinit of PyTerrier
  warn("Avoiding reinit of PyTerrier")
/opt/conda/lib/python3.10/site-packages/pyterrier/parallel.py:32: UserWarning: Av

18:49:53.895 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
18:49:54.033 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
18:49:54.046 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based mod

,name,nDCG@5,nDCG@10,nDCG@20,RR@5,RR@10,RR@20,P@5,P@10,P@20,R@5,R@10,R@20
0,PoolParallelTransformer(BR(DirichletLM)),0.225889,0.246574,0.272896,0.335158,0.347794,0.353938,0.1905,0.15720,0.126325,0.174259,0.255904,0.356129
1,PoolParallelTransformer(BR(DirichletLM)),0.230404,0.249907,0.277118,0.342675,0.355436,0.362027,0.1929,0.15860,0.127375,0.176089,0.256366,0.360037
2,PoolParallelTransformer(BR(DirichletLM)),0.230569,0.251122,0.277478,0.341592,0.355210,0.361679,0.1930,0.15945,0.127425,0.175140,0.259136,0.361121
3,PoolParallelTransformer(BR(DirichletLM)),0.230813,0.251941,0.277601,0.341383,0.355313,0.361468,0.1934,0.15945,0.127125,0.173726,0.259864,0.360472
